In [17]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import io
import re
import bs4 as bs
import urllib.request

In [18]:
pd.set_option('display.max_colwidth', -1)

In [19]:
with open('index_2018.json') as f:
    d = json.load(f)

In [20]:
filling_2018 = json_normalize(d['Filings2018'])

In [21]:
filling_2018['EIN'] = filling_2018['EIN'].astype(np.int64)

In [22]:
filling_2018['DateTime'] = pd.to_datetime(filling_2018['SubmittedOn'])

In [23]:
filling_2018.FormType.unique()

array(['990', '990PF', '990EZ'], dtype=object)

In [24]:
#filling_2018_dup.loc[filling_2018_dup.groupby('EIN').DateTime.idxmax()].he

In [25]:
filling_2018_rdup = filling_2018.loc[filling_2018.groupby('EIN').DateTime.idxmax()]

In [ ]:
#filling_2018 = filling_2018_dup.loc[filling_2018_dup.groupby('EIN').SubmittedOn.idxmax()]

In [ ]:
#filling_2018_d = filling_2018.EIN.where(filling_2018.EIN == 10777850 ); filling_2018_d

In [ ]:
#df_final.loc[(df_final['website'] != 'N/A')]

In [47]:
filling_2018_rdup.count()

DLN                 409840
EIN                 409840
FormType            409840
LastUpdated         409840
ObjectId            409840
OrganizationName    409840
SubmittedOn         409840
TaxPeriod           409840
URL                 409840
DateTime            409840
dtype: int64

In [41]:
f_2018_1 = filling_2018_rdup.loc[(filling_2018_rdup['EIN'] == 10586306)] 
#df_columns.loc[(df_columns['FormType'] != '990PF')]

In [42]:
f_2018_1.head()

,DLN,EIN,FormType,LastUpdated,ObjectId,OrganizationName,SubmittedOn,TaxPeriod,URL,DateTime
130095,93492044004388,10586306,990EZ,2018-03-14T23:04:38,201830449349200438,BARCEDA FAMILIES,2018-03-05,201706,https://s3.amazonaws.com/irs-form-990/201830449349200438_public.xml,2018-03-05


In [48]:
ntee_filter = pd.read_csv('merge_BMF.csv'); ntee_filter.count()

EIN            15100
NAME           15100
STREET         15100
CITY           15100
ZIP            15100
ACTIVITY       15100
NTEE_CD        8546 
REVENUE_AMT    9759 
dtype: int64

In [49]:
#df_merge = ntee_filter.merge(filling_2018_rdup, how='left',left_on='EIN', right_on='EIN');
df_merge = ntee_filter.merge(filling_2018_rdup, how='inner',on='EIN');

In [60]:
df_merge.FormType.unique()

array(['990EZ', '990', '990PF'], dtype=object)

In [61]:
df_merge_990_PF = df_merge.loc[(df_merge['FormType'] != '990PF')]

In [62]:
df_merge_990_PF.count()

EIN                 2544
NAME                2544
STREET              2544
CITY                2544
ZIP                 2544
ACTIVITY            2544
NTEE_CD             1756
REVENUE_AMT         2540
DLN                 2544
FormType            2544
LastUpdated         2544
ObjectId            2544
OrganizationName    2544
SubmittedOn         2544
TaxPeriod           2544
URL                 2544
DateTime            2544
dtype: int64

In [64]:
df_columns = df_merge_990_PF[['EIN','NAME', 'STREET', 'CITY', 'ZIP', 'ACTIVITY','NTEE_CD','URL']]; df_columns.count()

EIN         2544
NAME        2544
STREET      2544
CITY        2544
ZIP         2544
ACTIVITY    2544
NTEE_CD     1756
URL         2544
dtype: int64

In [65]:
df_columns.count()

EIN         2544
NAME        2544
STREET      2544
CITY        2544
ZIP         2544
ACTIVITY    2544
NTEE_CD     1756
URL         2544
dtype: int64

In [66]:
df_columns_na = df_columns.loc[(df_columns['URL'] != 'N/A')]

In [67]:
df_columns_na = df_columns_na.loc[(df_columns['URL'] != None )]

In [68]:
df_columns_na  = df_columns_na [df_columns_na ['URL'].notnull()]

In [71]:
df_columns_na.count()

EIN         2544
NAME        2544
STREET      2544
CITY        2544
ZIP         2544
ACTIVITY    2544
NTEE_CD     1756
URL         2544
dtype: int64

In [85]:
for index, row in df_columns_na.iterrows():
    url=row["URL"]
    
    sauce = urllib.request.urlopen(url).read()
    
    soup = bs.BeautifulSoup(sauce,'xml')
    
    phone = '' if soup.find('PhoneNum') == None or soup.find('PhoneNum').text == 'N/A' else soup.find('PhoneNum').text
    df_columns_na.loc[index, 'phone'] = phone
    
    website = '' if soup.find('WebsiteAddressTxt') == None or soup.find('WebsiteAddressTxt').text == 'N/A' or soup.find('WebsiteAddressTxt').text == 'NA'else soup.find('WebsiteAddressTxt').text
    df_columns_na.loc[index, 'website'] = website
    
    primary_except = '' if soup.find('PrimaryExemptPurposeTxt') == None or soup.find('PrimaryExemptPurposeTxt').text == 'N/A' else soup.find('PrimaryExemptPurposeTxt').text
    df_columns_na.loc[index, 'primary_except'] = primary_except
    
    DescriptionProgramSrvcAccomTxt = '' if soup.find('DescriptionProgramSrvcAccomTxt') == None or soup.find('DescriptionProgramSrvcAccomTxt').text == 'N/A' else soup.find('DescriptionProgramSrvcAccomTxt').text
    df_columns_na.loc[index, 'DescriptionProgramSrvcAccomTxt'] = DescriptionProgramSrvcAccomTxt
    

In [86]:
df_columns_na.head()

,EIN,NAME,STREET,CITY,ZIP,ACTIVITY,NTEE_CD,URL,phone,website,primary_except,DescriptionProgramSrvcAccomTxt
0,10605038,HABITAT FOR HUMANITY INTERNATIONAL INC,PO BOX 223,CASSVILLE,65625-0223,29380000,L20Z,https://s3.amazonaws.com/irs-form-990/201832499349200818_public.xml,4178472203,,SEE SCH O,TO IMPROVE THE QUALITY OF LIFE IN SOUTH BARRY COUNTY BY ELIMATING SUB-STANDARD HOUSING
1,15145247,AMVETS,318 BROAD ST,MOKANE,65059-1341,907000000,NaN,https://s3.amazonaws.com/irs-form-990/201833069349302108_public.xml,5736765494,,,
2,20731227,ROTARY INTERNATIONAL,3200 S STATE ROUTE 291,INDEPENDENCE,64057-1702,520349200,NaN,https://s3.amazonaws.com/irs-form-990/201811309349202611_public.xml,8168330078,EASTERNINDEPENDENCEROTARY.ORG,"ROTARY IS A SERVICE ORGANIZATION WHOSE MEMBERS WORK LOCALLY, REGIONALLY, AND INTERNATIONALLY THROUGH SERVICE PROJECTS AND DIRECT CHARITABLE CONTRIBUTIONS TO HELP THOSE LESS FORTUNATE THAN OURSELVES.","THE ROTARY CLUB OF INDEPENDENCE USED REVENUE FROM MEMBER DUES, MEALS, AND SPECIAL EVENTS TO FUND CHARITABLE CONTRIBUTIONS AND SERVICE PROJECTS INCLUDING CONTRIBUTIONS TO ROTARY INTERNATIONAL, INDEPENDENCE SCHOOL DISTRICT AND JOB ONE EMPLOYMENT OPPORTUNITIES FOR PEOPLE WITH DISABILITIES."
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,PO BOX 1327,COLUMBIA,65205-1327,179029000,Q113,https://s3.amazonaws.com/irs-form-990/201822849349300532_public.xml,5737772924,HTTP://FOLUS.ORG,,
4,133352209,INTERNATIONAL PEDIATRIC NEPHROLOGY ASSOCIATION HOSPITAL,2401 GILLHAM RD,KANSAS CITY,64108-4619,161179000,NaN,https://s3.amazonaws.com/irs-form-990/201823189349312167_public.xml,8162343812,IPNA-ONLINE.ORG,,


In [94]:
df_columns_website_nn  = df_columns_na[df_columns_na['website'] != ""]

In [107]:
df_columns_website_nn.count()

EIN                               1383
NAME                              1383
STREET                            1383
CITY                              1383
ZIP                               1383
ACTIVITY                          1383
NTEE_CD                           1072
URL                               1383
phone                             1383
website                           1383
primary_except                    1383
DescriptionProgramSrvcAccomTxt    1383
dtype: int64

In [108]:
df_columns_website_nn.head()

,EIN,NAME,STREET,CITY,ZIP,ACTIVITY,NTEE_CD,URL,phone,website,primary_except,DescriptionProgramSrvcAccomTxt
2,20731227,ROTARY INTERNATIONAL,3200 S STATE ROUTE 291,INDEPENDENCE,64057-1702,520349200,NaN,https://s3.amazonaws.com/irs-form-990/201811309349202611_public.xml,8168330078,EASTERNINDEPENDENCEROTARY.ORG,"ROTARY IS A SERVICE ORGANIZATION WHOSE MEMBERS WORK LOCALLY, REGIONALLY, AND INTERNATIONALLY THROUGH SERVICE PROJECTS AND DIRECT CHARITABLE CONTRIBUTIONS TO HELP THOSE LESS FORTUNATE THAN OURSELVES.","THE ROTARY CLUB OF INDEPENDENCE USED REVENUE FROM MEMBER DUES, MEALS, AND SPECIAL EVENTS TO FUND CHARITABLE CONTRIBUTIONS AND SERVICE PROJECTS INCLUDING CONTRIBUTIONS TO ROTARY INTERNATIONAL, INDEPENDENCE SCHOOL DISTRICT AND JOB ONE EMPLOYMENT OPPORTUNITIES FOR PEOPLE WITH DISABILITIES."
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,PO BOX 1327,COLUMBIA,65205-1327,179029000,Q113,https://s3.amazonaws.com/irs-form-990/201822849349300532_public.xml,5737772924,HTTP://FOLUS.ORG,,
4,133352209,INTERNATIONAL PEDIATRIC NEPHROLOGY ASSOCIATION HOSPITAL,2401 GILLHAM RD,KANSAS CITY,64108-4619,161179000,NaN,https://s3.amazonaws.com/irs-form-990/201823189349312167_public.xml,8162343812,IPNA-ONLINE.ORG,,
5,161608735,OUR LADY OF PEACE INC,12250 WEBER HILL RD STE 200,SAINT LOUIS,63127-1552,31150120,NaN,https://s3.amazonaws.com/irs-form-990/201723199349314777_public.xml,7162982900,www.ladyofpeace.org,,
6,200901499,MERCY HEALTH FOUNDATION,14528 SOUTH OUTER 40 RD STE 100,CHESTERFIELD,63017-5743,31150120,NaN,https://s3.amazonaws.com/irs-form-990/201801359349311630_public.xml,3145796100,www.mercy.net,,


In [127]:
df_columns_website_nn_filter = df_columns_website_nn[['EIN', 'NAME', 'website', 'phone', 'NTEE_CD']]

In [128]:
df_columns_website_nn_filter['EIN'] = df_columns_website_nn_filter['EIN'].astype(str).str.zfill(9)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [129]:
df_columns_website_nn_filter_rename = df_columns_website_nn_filter.rename(
    columns={'EIN': 'EIN', 'NAME': 'name', 'website': 'url', 'phone': 'phone','NTEE_CD': 'services' })

In [130]:
df_columns_website_nn_filter_rename.dtypes

EIN         object
name        object
url         object
phone       object
services    object
dtype: object

In [131]:
df_columns_website_nn_filter_rename.count()

EIN         1383
name        1383
url         1383
phone       1383
services    1072
dtype: int64

In [ ]:
df_columns_na_rename.reset_index(drop=True, inplace=True)

In [132]:
df_columns_website_nn_filter_rename.columns

Index(['EIN', 'name', 'url', 'phone', 'services'], dtype='object')

In [ ]:
#df_columns_na_rename = df_columns_na_rename.loc[(df_columns['url'] != 'N/A')]

In [152]:
df_columns_website_nn_filter_rename.head()

,EIN,name,url,phone,services
2,020731227,ROTARY INTERNATIONAL,EASTERNINDEPENDENCEROTARY.ORG,8168330078,NaN
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,HTTP://FOLUS.ORG,5737772924,Q113
4,133352209,INTERNATIONAL PEDIATRIC NEPHROLOGY ASSOCIATION HOSPITAL,IPNA-ONLINE.ORG,8162343812,NaN
5,161608735,OUR LADY OF PEACE INC,www.ladyofpeace.org,7162982900,NaN
6,200901499,MERCY HEALTH FOUNDATION,www.mercy.net,3145796100,NaN


In [148]:
df_columns_website_nn_filter_rename_lower['url_lower'] = df_columns_website_nn_filter_rename['url'].str.lower()
    #s.str.lower() df['x'].str.lower()data["First Name"]= data["First Name"].str.lower()df.url.str.lower()

In [155]:
df_columns_website_nn_filter_rename['url'] = df_columns_website_nn_filter_rename.url.str.lower()

In [156]:
df_columns_website_nn_filter_rename.head()

,EIN,name,url,phone,services
2,020731227,ROTARY INTERNATIONAL,easternindependencerotary.org,8168330078,NaN
3,131995940,LUDHIANA CHRISTIAN MEDICAL COLLEGE BOARD U S A INC,http://folus.org,5737772924,Q113
4,133352209,INTERNATIONAL PEDIATRIC NEPHROLOGY ASSOCIATION HOSPITAL,ipna-online.org,8162343812,NaN
5,161608735,OUR LADY OF PEACE INC,www.ladyofpeace.org,7162982900,NaN
6,200901499,MERCY HEALTH FOUNDATION,www.mercy.net,3145796100,NaN


In [157]:
df_columns_website_nn_filter_rename.to_csv('final_submission.csv')

In [158]:
df_columns_website_nn_filter_rename.to_json('final_submission.json', orient = "records")

In [159]:
df_columns_website_nn_filter_rename.count()

EIN         1383
name        1383
url         1383
phone       1383
services    1072
dtype: int64